In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

## Notebook này sẽ giải thích từng line code 1 của từng def

In [5]:
interpreter = tf.lite.Interpreter(model_path=r"single_pose.tflite")
interpreter.allocate_tensors()

In [6]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    #reshape the frame to 1x192x192x3
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()


    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)

    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.4571607  0.40652776 0.69493556]
   [0.41019887 0.47348484 0.73562944]
   [0.4112266  0.38901728 0.75919634]
   [0.45445776 0.5829029  0.5699538 ]
   [0.43974918 0.39518225 0.48571637]
   [0.677474   0.73597944 0.67629635]
   [0.6465199  0.27765507 0.7478485 ]
   [0.65560466 0.81647134 0.25130293]
   [0.878539   0.18884437 0.17054994]
   [0.4989124  0.7087898  0.19433822]
   [0.6127289  0.26477918 0.01483351]
   [0.999156   0.6336676  0.00518921]
   [0.9862966  0.34516823 0.00824883]
   [0.4505112  0.5248575  0.031262  ]
   [0.662263   0.25934732 0.01919806]
   [0.4594128  0.6818924  0.10171992]
   [0.7272248  0.03074136 0.02905555]]]]


In [8]:
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]
print(f"right_eye: {right_eye}")
print("----------------")
print(f"left_elbow: {left_elbow}")

right_eye: [0.4112266  0.38901728 0.75919634]
----------------
left_elbow: [0.65560466 0.81647134 0.25130293]


In [10]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1])) #(480,640) là chiều cao, chiều dài của ảnh
for kp in shaped:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)

219 260 0.6949355602264404
196 303 0.7356294393539429
197 248 0.7591963410377502
218 373 0.5699537992477417
211 252 0.4857163727283478
325 471 0.6762963533401489
310 177 0.7478485107421875
314 522 0.25130292773246765
421 120 0.17054994404315948
239 453 0.19433821737766266
294 169 0.014833508990705013
479 405 0.005189213901758194
473 220 0.008248832076787949
216 335 0.03126199543476105
317 165 0.019198056310415268
220 436 0.1017199233174324
349 19 0.029055554419755936


In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #radius = 4, color = (0,255,0), thickness = -1

In [11]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [13]:
shaped[0], shaped[1]
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = shaped[p1]
    y2, x2, c2 = shaped[p2]
    print((int(x2), int(y2)))

(303, 196)
(248, 197)
(373, 218)
(252, 211)
(471, 325)
(177, 310)
(522, 314)
(453, 239)
(120, 421)
(169, 294)
(177, 310)
(405, 479)
(220, 473)
(220, 473)
(335, 216)
(436, 220)
(165, 317)
(19, 349)


In [14]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)